In [ ]:
# # !pip install boto3
# # !pip install python-dotenv
# !pip install --upgrade boto3 botocore urllib3


In [1]:
import random
import uuid
import importlib
from pathlib import Path
from datetime import date, timedelta

import run_services as rn
import flights.aggregator as ag
import media.video_generator as vg
import content.caption_builder as cb
import review.telegram_review as tr
from instagram.ig_client import InstagramClient
import affiliates.affiliates as af
import web.exporter as ex
import web.uploader as up
from flights.base import Flight
from flights.published_history import is_recently_published, register_publication


LOGO_PATH = "media/images/EscapGo_circ_logo_transparent.png"
VIDEO_PATH = "media/videos/reel.mp4"
MARKET = "PMI"


# ----------------------------------------------
# 1) Ventana aleatoria: hoy → random(2–6 meses)
# ----------------------------------------------
def choose_random_search_window(min_months=2, max_months=2):
    today = date.today()
    months = random.randint(min_months, max_months)
    offset_days = months * 30
    end = today + timedelta(days=offset_days)
    return today, end


# ----------------------------------------------
# 2) Elegir main candidate
# ----------------------------------------------
def pick_main_candidate(min_discount_pct=40.0):
    start, end = choose_random_search_window()
    print(f"🔎 Buscando vuelos entre {start} y {end}")

    flights = ag.get_flights_in_period(start, end)
    print(f"   {len(flights)} vuelos encontrados")

    flights = [f for f in flights if not is_recently_published(f)]
    print(f"   {len(flights)} tras filtrar publicados")

    if not flights:
        raise RuntimeError("No hay vuelos nuevos")

    best_by_cat = ag.get_best_by_category_scored(
        flights, min_discount_pct=min_discount_pct
    )

    if not best_by_cat:
        raise RuntimeError("No hay vuelos con descuento suficiente")

    main_item = ag.choose_main_candidate_prob(best_by_cat)
    return main_item, best_by_cat, flights


# ----------------------------------------------
# 3–4) Generar VIDEO + CAPTION
# ----------------------------------------------
def build_video_and_caption(main_item):
    main_flight: Flight = main_item["flight"]
    main_category_code = (
        main_item.get("category_code")
        or main_item.get("category", {}).get("code")
    )

    # 3) CAPTION
    importlib.reload(cb)
    caption_text = cb.build_caption_for_flight(
        main_flight,
        category_code=main_category_code,
        tone="emocional",
    )

    # 4) VIDEO (local para review)
    importlib.reload(vg)
    vg.create_reel_for_flight(
        main_flight,
        out_mp4_path=VIDEO_PATH,
        logo_path=LOGO_PATH,
        duration=4.0,
        s3_bucket=None,      # <--- NO subir aún
    )

    return main_flight, main_category_code, caption_text


# ----------------------------------------------
# 5) Enviar a review Telegram (local mp4)
#   👉 Aquí reordenamos candidatos para que el main
#      quede SIEMPRE en índice 0.
# ----------------------------------------------
def send_to_review(main_item, best_by_cat, caption_text):
    main_flight: Flight = main_item["flight"]
    main_cat_code = (
        main_item.get("category_code")
        or main_item.get("category", {}).get("code")
    )

    job_id = str(uuid.uuid4())

    # 5.1) Convertimos todos a candidatos para Telegram
    review_candidates = tr.to_review_candidates(best_by_cat)

    # 5.2) Buscamos qué índice de best_by_cat corresponde al main
    main_idx = 0
    for i, item in enumerate(best_by_cat):
        item_flight = item["flight"]
        item_cat_code = (
            item.get("category_code")
            or item.get("category", {}).get("code")
        )
        if item_flight is main_flight and item_cat_code == main_cat_code:
            main_idx = i
            break

    # 5.3) Reordenamos para que el principal quede en posición 0
    if main_idx != 0:
        print(f"ℹ️ Reordenando candidatos: main_idx={main_idx} pasa a 0")
        review_candidates[0], review_candidates[main_idx] = (
            review_candidates[main_idx],
            review_candidates[0],
        )

    # 5.4) Registrar job: current_index arranca en 0 -> main candidate
    tr.register_job(
        job_id=job_id,
        flight=main_flight,
        caption=caption_text,
        video_path=Path(VIDEO_PATH),
        candidates=review_candidates,
    )

    tr.send_review_candidate(job_id)
    print(f"📲 Enviado a review. job_id={job_id}")

    return job_id


# ----------------------------------------------
# 6–7–8) Publicar en IG, actualizar web, registrar histórico
# (esta función queda para modo auto_publish futuro)
# ----------------------------------------------
def publish_to_instagram_and_update_web(main_item, caption_text):
    main_flight: Flight = main_item["flight"]

    print("📤 Subiendo vídeo a S3 para Instagram...")
    video_url = vg.upload_reel_to_s3(
        VIDEO_PATH,
        bucket="escapadasgo-reels",
        prefix="pmi/",
    )

    print("📤 Publicando en Instagram...")
    ig = InstagramClient()

    creation_id = ig.create_reel_container(video_url=video_url, caption=caption_text)
    if not ig.wait_until_ready(creation_id):
        print("❌ Instagram no procesó el vídeo")
        return

    reel_id = ig.publish_reel(creation_id)
    permalink = ig.get_media_permalink(reel_id)
    print(f"   ✔ Publicado: {permalink}")

    print("🗂 Actualizando web...")
    affiliate_url = af.build_affiliate_url_for_flight(main_flight)

    data = ex.update_flights_json(
        main_item=main_item,
        json_path="local_copy.json",
        market=MARKET,
        reel_url=permalink,
        affiliate_url=affiliate_url,
        max_entries=5,
    )

    up.upload_flights_json(data, key=f"{MARKET.lower()}/flights_of_the_day.json")
    print("   ✔ Web actualizada")

    register_publication(main_flight, category_code=main_item.get("category_code"))
    print("📝 Histórico actualizado")


# # ----------------------------------------------
# # Orquestador principal
# # ----------------------------------------------
# if __name__ == "__main__":
#     print("🚀 Lanzando daily workflow Escapadas GO (Mallorca)...")

#     # Si quisieras levantar servicios:
#     # print("🔧 Levantando servicios (web + Telegram)...")
#     # proc = rn.start_services()

#     min_discount_pct = 40
#     auto_publish = False  # de momento sólo modo review

#     try:
#         # 2) Buscar vuelos y elegir main candidate
#         main_item, best_by_cat, flights = pick_main_candidate(
#             min_discount_pct=min_discount_pct
#         )
#         main_flight: Flight = main_item["flight"]
#         print(
#             f"✅ Candidato principal: {main_flight.origin} → {main_flight.destination} "
#             f"{main_flight.start_date[:10]} – {main_flight.end_date[:10]} "
#             f"@ {getattr(main_flight, 'price', '?')} €"
#         )

#         # 3 + 4) Vídeo + caption
#         main_flight, main_category_code, caption_text = build_video_and_caption(
#             main_item
#         )

#         # 5) Mandar a review (flujo actual)
#         job_id = send_to_review(main_item, best_by_cat, caption_text)

#         if auto_publish:
#             # FUTURO: publicación automática sin review
#             publish_to_instagram_and_update_web(main_item, caption_text)
#         else:
#             print(
#                 "ℹ️ Modo revisión activado: la publicación en IG + S3 se hará desde el flujo de Telegram."
#             )

#     finally:
#         # Aquí podrías cerrar servicios si usas rn.start_services()
#         pass

In [ ]:
# print("📤 Subiendo vídeo a S3 para Instagram...")
# s3 = boto3.client("s3", region_name="eu-north-1")
# video_url = vg.upload_reel_to_s3(
#     VIDEO_PATH,
#     bucket="escapadasgo-reels",
#     prefix="pmi/",
# )
# print(video_url)

In [ ]:
# rn.stop_services(proc)

In [2]:
# 1) Levantar servicios web / bot
print("🔧 Levantando servicios (web + Telegram)...")
proc = rn.start_services()  # si devuelve algo tipo Popen, queda en background


🔧 Levantando servicios (web + Telegram)...
Servicios iniciados (PID 6524)


In [3]:
    print("🚀 Lanzando daily workflow Escapadas GO (Mallorca)...")

    # Si quisieras levantar servicios:
    # print("🔧 Levantando servicios (web + Telegram)...")
    # proc = rn.start_services()

    min_discount_pct = 40
    auto_publish = False  # de momento sólo modo review

    try:
        # 2) Buscar vuelos y elegir main candidate
        main_item, best_by_cat, flights = pick_main_candidate(
            min_discount_pct=min_discount_pct
        )
        main_flight: Flight = main_item["flight"]
        print(
            f"✅ Candidato principal: {main_flight.origin} → {main_flight.destination} "
            f"{main_flight.start_date[:10]} – {main_flight.end_date[:10]} "
            f"@ {getattr(main_flight, 'price', '?')} €"
        )

        # 3 + 4) Vídeo + caption
        main_flight, main_category_code, caption_text = build_video_and_caption(
            main_item
        )

        # 5) Mandar a review (flujo actual)
        job_id = send_to_review(main_item, best_by_cat, caption_text)

        if auto_publish:
            # FUTURO: publicación automática sin review
            publish_to_instagram_and_update_web(main_item, caption_text)
        else:
            print(
                "ℹ️ Modo revisión activado: la publicación en IG + S3 se hará desde el flujo de Telegram."
            )

    finally:
        # Aquí podrías cerrar servicios si usas rn.start_services()
        pass

🚀 Lanzando daily workflow Escapadas GO (Mallorca)...
🔎 Buscando vuelos entre 2025-12-08 y 2026-02-06
🔎 Buscando vuelos entre 2025-12-08 y 2026-02-06...
🗓  Buscando vuelos 2025-12-11 → 2025-12-14...
🗓  Buscando vuelos 2025-12-11 → 2025-12-15...
🗓  Buscando vuelos 2025-12-12 → 2025-12-14...
🗓  Buscando vuelos 2025-12-12 → 2025-12-15...
🗓  Buscando vuelos 2025-12-18 → 2025-12-21...
🗓  Buscando vuelos 2025-12-18 → 2025-12-22...
🗓  Buscando vuelos 2025-12-19 → 2025-12-21...
🗓  Buscando vuelos 2025-12-19 → 2025-12-22...
🗓  Buscando vuelos 2025-12-25 → 2025-12-28...
🗓  Buscando vuelos 2025-12-25 → 2025-12-29...
🗓  Buscando vuelos 2025-12-26 → 2025-12-28...
🗓  Buscando vuelos 2025-12-26 → 2025-12-29...
🗓  Buscando vuelos 2026-01-01 → 2026-01-04...
🗓  Buscando vuelos 2026-01-01 → 2026-01-05...
🗓  Buscando vuelos 2026-01-02 → 2026-01-04...
🗓  Buscando vuelos 2026-01-02 → 2026-01-05...
🗓  Buscando vuelos 2026-01-08 → 2026-01-11...
🗓  Buscando vuelos 2026-01-08 → 2026-01-12...
🗓  Buscando vuelos 2

Moviepy - Done !
Moviepy - video ready media/videos/reel.mp4
ℹ️ Reordenando candidatos: main_idx=1 pasa a 0
📲 Enviado a review. job_id=2cb8ee6f-f3e2-4dff-a701-04594f66acf9
ℹ️ Modo revisión activado: la publicación en IG + S3 se hará desde el flujo de Telegram.


In [ ]:
best_by_cat

In [ ]:
import pandas as pd
df_flights = pd.DataFrame(flights)

In [ ]:
df_flights.sort_values(by='price')

In [ ]:
df_flights[df_flights['category_code']=='finde_perfecto'].sort_values(by='price')

In [ ]:
df_flights[df_flights['destination']=='VIE'].sort_values(by='price')

In [ ]:
len(df_flights[df_flights['destination']=='VIE'])

In [ ]:
df_flights[df_flights['destination']=='VIE'].price.mean()

In [ ]:
import numpy as np
# a = [154, 400, 1124, 82, 94, 108]
np.percentile(df_flights[df_flights['destination']=='VIE'].price,70) # gives the 95th percentile

In [ ]:
import importlib

import run_services as rn

# flights
import flights.aggregator as ag
from datetime import date, timedelta

# video
import media.video_generator as vg

# caption
import content.caption_builder as cb

# instagram
from pathlib import Path
import os
from instagram.ig_client import InstagramClient

# review
import uuid
from pathlib import Path
import review.telegram_review as tr

import affiliates.affiliates as af


In [ ]:
proc = rn.start_services()

In [ ]:
# rn.stop_services(proc)

## FLIGHTS

In [ ]:
import pandas as pd

In [ ]:
importlib.reload(ag)


In [ ]:
start = date.today()
end = start + timedelta(days=15)

flights = ag.get_flights_in_period(start, end)
best_by_cat = ag.get_best_by_category_scored(flights, min_discount_pct=40.0)
# best_by_cat = ag.get_best_by_category_scored(flights[25:35])

for item in best_by_cat:
    f = item["flight"]
    cat = item["category"]
    score = item["score"]
    print(cat["label"], f"{f.origin}→{f.destination}", f.price, f.price_per_km, score)

In [ ]:
main_item = ag.choose_main_candidate_prob(best_by_cat)

# main_item = best_by_cat[1]

if main_item is None:
    raise ValueError("No hay candidatos para hoy")

main_flight = main_item["flight"]
main_category_code = main_item["category"]["code"]

main_flight

In [ ]:
NGROK_BASE_URL = "https://karie-apocatastatic-maddox.ngrok-free.dev" 
BASE_DIR = "/media/videos/reel.mp4"
No_cache = "?cache_bust=1"

video_url = NGROK_BASE_URL + BASE_DIR + No_cache
video_url

In [ ]:
from web.exporter import update_flights_json

import affiliates.affiliates as af
importlib.reload(af)

import web.exporter as ex
import web.uploader as up

importlib.reload(ex)
importlib.reload(up)




flight = main_item["flight"]   # tu Flight(...)
affiliate_url = af.build_affiliate_url_for_flight(flight)

data = ex.update_flights_json(
    main_item=main_item,
    json_path="local_copy.json",  # opcional
    market="PMI",
    reel_url=video_url,
    affiliate_url=affiliate_url,
    max_entries=5,
)

up.upload_flights_json(data, key="pmi/flights_of_the_day.json")


In [ ]:
affiliate_url

## VIDEO

In [ ]:
# from flights.base import Flight   # o desde donde lo tengas

# main_flight = Flight(
#     origin='PMI',
#     destination='LPA',
#     price=74.23,
#     start_date='2025-12-11 09:20:00',
#     end_date='2025-12-15 13:30:00',
#     airline='Ryanair',
#     link='https://www.ryanair.com/es/es/trip/flights/select?...',
#     distance_km=837.070500454846,
#     price_per_km=0.0886783131882739,
#     route_typical_price=126.60,
#     discount_pct=41.4,
#     category_code='cultural',
#     category_label='Escapada Cultural'
# )

In [ ]:
importlib.reload(vg)

logo_path = "media/images/EscapGo_circ_logo_transparent.png"   # circular, transparente
out_path = "media/videos/reel.mp4"

vg.create_reel_for_flight(
    main_flight,               # tu objeto Flight o dict
    out_mp4_path=out_path,
    logo_path=logo_path,
    duration=4.0,
)

## CAPTION

In [ ]:
importlib.reload(cb)
caption_text = cb.build_caption_for_flight(
    main_flight,
    category_code=main_category_code,  # "cultural", "romantica", etc.
    tone="emocional",
)

caption_text

## REVIEW

In [ ]:
# Después de elegir tu "mejor vuelo", generar caption y vídeo:
job_id = str(uuid.uuid4())

local_video_path = Path("media/videos/reel.mp4")

review_candidates = tr.to_review_candidates(best_by_cat)

tr.register_job(
    job_id=job_id,
    flight=best_by_cat[0]['flight'],
    caption=caption_text,
    video_path=local_video_path,
    candidates=review_candidates,  # si quieres pasarle más opciones
)

tr.send_review_candidate(job_id)
print(f"Enviado a Telegram para revisión. job_id={job_id}")

## INSTAGRAM

In [ ]:
video_url

In [ ]:
caption_text = "Test Reel desde @escapadas_mallorca 🚀"

# 4) Cliente IG
ig = InstagramClient()

# 5) Crear contenedor
creation_id = ig.create_reel_container(video_url=video_url, caption=caption_text)

# 6) Esperar a que esté listo
if ig.wait_until_ready(creation_id):
    # 7) Publicar
    reel_id = ig.publish_reel(creation_id)
    permalink = ig.get_media_permalink(reel_id)
    print("✅ Reel publicado:")
    print("ID:", reel_id)
    print("Permalink:", permalink)
else:
    print("❌ No se pudo procesar el vídeo, no se publica.")

In [ ]:
import pandas as pd
pd.DataFrame(flights).sort_values(by='price')

In [ ]:
# main.py

import time
from datetime import date

from config.settings import SETTINGS
from flights.aggregator import get_best_daily_flight
from content.caption_builder import build_caption
from media.image_picker import pick_image_for_destination
from media.video_generator import generate_reel_video
from storage.uploader import upload_video_and_get_url
from instagram.publish import publish_reel
from review.telegram_bot import send_review_message, wait_for_user_decision

In [ ]:
def daily_workflow():
    print("🚀 Iniciando flujo diario...")

    # 1) Obtener mejor oferta del día (de cualquier API disponible)
    print("🔍 Buscando mejor vuelo del día...")
    flight = get_best_daily_flight(date.today())
    if not flight:
        print("❌ No se encontró ningún vuelo.")
        return

    # 2) Generar caption con OpenAI (JSON estrictamente controlado)
    print("✍️ Generando caption...")
    caption_json = build_caption(flight)
    caption_text = caption_json_to_string(caption_json)  # si necesitas convertir

    # 3) Seleccionar imagen asociada al destino
    print("🖼 Seleccionando imagen...")
    image_path = pick_image_for_destination(flight.destination)

    # 4) Generar vídeo estilo Reel (6s, zoom, texto)
    print("🎬 Generando vídeo...")
    video_local_path = generate_reel_video(image_path, flight)

    # 5) Enviar todo a Telegram para revisión manual
    print("📲 Enviando a Telegram para revisión...")
    send_review_message(flight, caption_text, video_local_path)

    # 6) Esperar la decisión humana (bloqueante o async)
    print("⌛ Esperando decisión del usuario...")
    decision = wait_for_user_decision()

    if decision == "approve":
        print("👍 Usuario aprobó. Publicando en Instagram...")

        # 7) Subir vídeo a una URL pública
        print("🌐 Subiendo vídeo...")
        video_url = upload_video_and_get_url(video_local_path)

        # 8) Publicar en Instagram
        print("📤 Publicando Reel...")
        permalink = publish_reel(video_url, caption_text)

        print("🎉 Publicado con éxito:", permalink)
        return permalink

    elif decision == "reject":
        print("❌ Usuario rechazó el vuelo. Buscando otra oferta...")
        # Recursivo o bucle para repetir
        return daily_workflow()

    else:
        print("⚠️ Decisión desconocida:", decision)


def caption_json_to_string(caption_json: dict) -> str:
    """
    Convierte el JSON de OpenAI al caption final formateado.
    """
    return (
        f"{caption_json['hook']}\n\n"
        f"{caption_json['summary_line']}\n\n"
        f"{caption_json['dates_block']}\n\n"
        f"{caption_json['itinerary_block']}\n\n"
        f"{caption_json['tips_block']}\n\n"
        f"{caption_json['cta_block']}\n\n"
        f"{caption_json['hashtags']}"
    )

In [ ]:

if __name__ == "__main__":
    daily_workflow()

In [ ]:
# import subprocess
# import sys
# import os
# import time
# import signal

# BASE_DIR = r"C:\Users\Macia\Desktop\Jupyter Notebooks\EscapadasMallorca"

# def start_services():
#     cmd = [sys.executable, "run_services.py"]

#     # Lanzamos el proceso, pero sin bloquear el hilo principal
#     proc = subprocess.Popen(
#         cmd,
#         cwd=BASE_DIR,
#         stdout=subprocess.PIPE,
#         stderr=subprocess.PIPE,
#         text=True
#     )
#     print(f"Servicios iniciados (PID {proc.pid})")
#     return proc

# def stop_services(proc):
#     print(f"Deteniendo servicios (PID {proc.pid})...")

#     # En Windows, Popen.terminate() envía una señal de terminación compatible
#     proc.terminate()

#     try:
#         proc.wait(timeout=10)
#         print("Servicios detenidos correctamente.")
#     except subprocess.TimeoutExpired:
#         print("Forzando cierre...")
#         proc.kill()

# if __name__ == "__main__":
#     proc = start_services()

#     print("\nPresiona ENTER para detener todo.\n")
#     input()

#     stop_services(proc)